In [22]:
import numpy as np
from scipy.linalg import lstsq
path = "D:/7_30data/"

In [23]:
import numpy as np
from scipy.linalg import lstsq

def iterative_refinement_solve(V, L_rho, max_iter=25, tolerance=1e-15, verbose=True):
    """
    高精度迭代精化求解复数线性系统 S @ a = b，其中 S = V^H @ V, b = V^H @ L_rho
    专门追求机器精度，允许更多迭代次数，更严格的收敛标准

    Parameters:
    -----------
    V : ndarray, shape (m, n)
        输入复数矩阵
    L_rho : ndarray, shape (m,)
        右端复数向量
    max_iter : int, default=25
        最大迭代次数（增加到25次以确保充分迭代）
    tolerance : float, default=1e-15
        收敛容差（机器精度级别）
    verbose : bool, default=True
        是否输出详细信息

    Returns:
    --------
    a : ndarray
        解向量（如果虚部可忽略则返回实数，否则返回复数）
    """

    # 确保输入数据类型为complex128以获得最高精度
    V = V.astype(np.complex128)
    L_rho = L_rho.astype(np.complex128)

    # 构建正规方程 S @ a = b
    S = V.conj().T @ V  # V^H @ V
    b = V.conj().T @ L_rho  # V^H @ L_rho

    # 计算矩阵的秩信息
    rank_S = np.linalg.matrix_rank(S)
    augmented = np.hstack((S, b.reshape(-1, 1)))
    rank_augmented = np.linalg.matrix_rank(augmented)
    print(f"rank(S) = {rank_S}")
    print(f"rank([S | b]) = {rank_augmented}")

    if verbose:
        # 输出条件数信息用于诊断
        cond_S = np.linalg.cond(S)
        print(f"Condition number of S: {cond_S:.4e}")
        print("\n=== High Precision Iterative Refinement ===")

    # 检查系统的相容性
    if rank_S != rank_augmented:
        if verbose:
            print("Warning: System is inconsistent (rank(S) ≠ rank([S|b]))")
            print("Using least squares solution...")

    # 步骤1: 获得初始解（使用最高精度的最小二乘法）
    try:
        a = lstsq(S, b, lapack_driver='gelsy')[0]
        if verbose:
            print("Initial solution computed using high-precision least squares")
    except Exception as e:
        if verbose:
            print(f"Failed to compute initial solution: {e}")
        return np.zeros(S.shape[1], dtype=np.complex128)

    # 计算初始残差
    initial_residual = np.linalg.norm(S @ a - b)
    if verbose:
        print(f"Initial residual norm: {initial_residual:.16e}")

    # 迭代精化过程 - 改进的停滞检测逻辑
    best_residual = initial_residual
    best_solution = a.copy()

    # 记录最近几次残差用于更智能的停止判断
    residual_history = []

    for i in range(max_iter):
        # 步骤2: 计算当前残差 r = b - S @ a
        r = b - S @ a
        residual_norm = np.linalg.norm(r)
        residual_history.append(residual_norm)

        if verbose:
            print(f"Iteration {i+1:2d}: Residual norm = {residual_norm:.16e}")

        # 记录最佳解
        if residual_norm < best_residual:
            best_residual = residual_norm
            best_solution = a.copy()

        # 步骤3: 严格的收敛检查
        if residual_norm <= tolerance:
            if verbose:
                print(f"Converged to tolerance after {i+1} iterations")
            break

        # 步骤4: 求解修正方程 S @ da = r
        try:
            da = lstsq(S, r, lapack_driver='gelsy')[0]
        except Exception as e:
            if verbose:
                print(f"Iteration {i+1} failed: {e}")
            break

        # 步骤5: 更新解 a = a + da
        a = a + da

        # 改进的停滞检测：只有在残差明显停滞且已经尝试足够多次时才停止
        if i >= 15:  # 至少迭代15次
            # 检查最近5次迭代的残差变化
            if len(residual_history) >= 5:
                recent_residuals = residual_history[-5:]
                max_recent = max(recent_residuals)
                min_recent = min(recent_residuals)

                # 如果最近5次残差的相对变化小于1%，且远大于机器精度，则认为停滞
                relative_change = (max_recent - min_recent) / min_recent if min_recent > 0 else 0

                if (relative_change < 0.01 and
                    min_recent > 100 * tolerance and
                    i >= 20):  # 至少20次迭代后才考虑真正停滞
                    if verbose:
                        print(f"Stopping: Residual stagnated (relative change: {relative_change:.2e})")
                        print("Using best solution found so far")
                    a = best_solution.copy()
                    break

    # 计算最终残差（确保使用最佳解）
    a = best_solution.copy()
    final_residual = np.linalg.norm(S @ a - b)

    if verbose:
        print(f"\nFinal residual norm for S @ a - b: {final_residual:.16e}")

        # 输出改进情况
        improvement_factor = initial_residual / final_residual if final_residual > 0 else float('inf')
        print(f"Improvement factor: {improvement_factor:.2e}")

        # 检查解的性质
        max_imag = np.max(np.abs(np.imag(a)))
        if max_imag > 1e-12:
            print(f"Solution has complex components (max imaginary part: {max_imag:.4e})")
        else:
            print("Solution is effectively real (converting to real)")

    # 输出必要的最终信息（不截断，显示完整精度）
    print(f"Residual norm for S @ a - b: {final_residual:.16e}")

    # 智能返回类型：如果虚部可忽略则返回实数
    max_imag = np.max(np.abs(np.imag(a)))
    if max_imag < 1e-12:
        return np.real(a)
    else:
        return a

In [24]:
# 使用示例_1
V = np.load(path+'V.npy').astype(np.complex128)
L_rho = np.load(path+'L_rho.npy').astype(np.complex128)
x = iterative_refinement_solve(V, L_rho,verbose=True)
print("\n")
y=iterative_refinement_solve(V, L_rho,verbose=False)

rank(S) = 2
rank([S | b]) = 2
Condition number of S: 9.7122e+80

=== High Precision Iterative Refinement ===
Initial solution computed using high-precision least squares
Initial residual norm: 1.4461906702066530e-10
Iteration  1: Residual norm = 1.4461906702066530e-10
Iteration  2: Residual norm = 2.0579515875677572e-11
Iteration  3: Residual norm = 8.2318063498103511e-11
Iteration  4: Residual norm = 2.0579515875517370e-11
Iteration  5: Residual norm = 2.0579515875476124e-11
Iteration  6: Residual norm = 2.0579515875463642e-11
Iteration  7: Residual norm = 2.1084465186937517e-16
Converged to tolerance after 7 iterations

Final residual norm for S @ a - b: 2.1084465186937517e-16
Improvement factor: 6.86e+05
Solution is effectively real (converting to real)
Residual norm for S @ a - b: 2.1084465186937517e-16


rank(S) = 2
rank([S | b]) = 2
Residual norm for S @ a - b: 2.1084465186937517e-16


In [25]:
# 使用示例_2
np.random.seed(42)
V = np.random.randn(4, 273) + 1j * np.random.randn(4, 273)
L_rho = np.random.randn(4) + 1j * np.random.randn(4)
solution = iterative_refinement_solve(V, L_rho, verbose=True)
solution = iterative_refinement_solve(V, L_rho, verbose=False)

# 使用示例_3
np.random.seed(100)
V = np.random.randn(4, 273) + 1j * np.random.randn(4, 273)
L_rho = np.random.randn(4) + 1j * np.random.randn(4)
solution = iterative_refinement_solve(V, L_rho, verbose=True)
solution = iterative_refinement_solve(V, L_rho, verbose=False)

rank(S) = 4
rank([S | b]) = 4
Condition number of S: 1.8539e+19

=== High Precision Iterative Refinement ===
Initial solution computed using high-precision least squares
Initial residual norm: 2.8866405161597366e-14
Iteration  1: Residual norm = 2.8866405161597366e-14
Iteration  2: Residual norm = 1.7600757088117352e-14
Iteration  3: Residual norm = 1.7711692175792960e-14
Iteration  4: Residual norm = 1.7279235285330439e-14
Iteration  5: Residual norm = 1.8574849046989312e-14
Iteration  6: Residual norm = 1.9666570987670721e-14
Iteration  7: Residual norm = 1.8971336933154305e-14
Iteration  8: Residual norm = 1.8019034268696099e-14
Iteration  9: Residual norm = 1.7507053463922958e-14
Iteration 10: Residual norm = 1.8728776492714919e-14
Iteration 11: Residual norm = 1.8043750063518024e-14
Iteration 12: Residual norm = 1.7860327410391217e-14
Iteration 13: Residual norm = 1.7866464740696191e-14
Iteration 14: Residual norm = 1.8976086106618789e-14
Iteration 15: Residual norm = 1.8901392880

In [26]:
# 测试
# 设置测试组数量
num_test_cases = 20

# 使用循环一次生成多组数据并测试函数
for i in range(num_test_cases):
    print(f"--- test {i+1} ---")
    # 为每个测试用例设置不同的随机数种子，以确保数据独立且可复现
    rng = np.random.default_rng(seed=57 + i)
    # 使用当前的 rng 对象生成 V 和 L_rho
    V = rng.standard_normal(size=(4, 273))+ 1j * rng.standard_normal(size=(4, 273))
    L_rho = rng.standard_normal(size=(4)) + 1j * rng.standard_normal(size=(4))
    # 调用函数并测试
    try:
        solution_silent = iterative_refinement_solve(V, L_rho, verbose=False)
    except Exception as e:
        print(f"  test {i+1} ，error : {e}")
    print("-" * 30 + "\n")

--- test 1 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 1.2336317654586270e-14
------------------------------

--- test 2 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 7.4177534466143325e-15
------------------------------

--- test 3 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 1.7266656366101805e-14
------------------------------

--- test 4 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 1.0173815405343010e-14
------------------------------

--- test 5 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 1.9385244789896381e-14
------------------------------

--- test 6 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 1.0067997907628746e-14
------------------------------

--- test 7 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 1.2900318220698719e-14
------------------------------

--- test 8 ---
rank(S) = 4
rank([S | b]) = 4
Residual norm for S @ a - b: 8.7698548603048468e-15
